<a href="https://colab.research.google.com/github/francescomazzolin/RTS2/blob/main/IBM_HI_small_FE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Feature Engeneering**
---------------------

Behavior of the account (history of the account as sender/receiver) and behaviour of the pair of accounts.

Creation of features for both behaviour.

# Preparatory work

In [1]:
#Data ingestion
import pickle
import gzip

#General purpose
import pandas as pd
import numpy as np
import scipy.stats as stats
import math


#Plotting styles
import matplotlib.pyplot as plt
# from matplotlib_venn import venn3
# from matplotlib_venn import venn2
# import matplotlib_venn as venn
# import matplotlib.patches as mpatches
import seaborn as sns

#Graph Part
import networkx as nx

#Other
import time
import re
import os
import warnings
warnings.filterwarnings('ignore')
# import imageio

### Ingesting data

In [2]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


Francesco

In [3]:
# df_augmented = pd.read_pickle('drive/MyDrive/parsed_ibm_aml.pkl')

file_path = "/content/drive/My Drive/HI_small_with_encodings.pkl.gz"

with gzip.open(file_path, "rb") as f:
    df_augmented = pickle.load(f)


Giada

In [ ]:
file_path = r"G:\My Drive\HI_parsed_ibm_aml.pkl.gz"

with gzip.open(file_path, "rb") as f:
    df_augmented = pickle.load(f)

In [ ]:
os.listdir('/content/drive/MyDrive')

['Curriculum.gdoc',
 'Colab Notebooks',
 'presentation.pdf',
 'tfinance',
 'STOCHASTIC_FINANCE_PROJECT_modified.pdf',
 'STOCHASTIC_FINANCE_PROJECT_modified.gdoc',
 'data_returns_correct.csv',
 'MLforFD',
 'tesi',
 'LI-Small.pkl',
 'SAML-D.csv']

In [ ]:
file_path = "/content/drive/My Drive/HI_parsed_ibm_aml.pkl.gz"

with gzip.open(file_path, "rb") as f:
    df_augmented = pickle.load(f)

In [ ]:
df_augmented.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,type_fraud,fraud_id,pattern_number,depth_info,depht_type_info,forex_multiple_Amount Paid,Normaized_Amount Paid,forex_multiple_Amount Received,Normaized_Amount Received
0,2022-09-01,121,8123FB9B0,121,8123FB9B0,47.639999,Saudi Riyal,47.639999,Saudi Riyal,Reinvestment,0,NaN,NaN,NaN,NaN,NaN,0.266392,12.690906,0.266392,12.690906
1,2022-09-01,2454,802E97980,2454,802E97980,14.630000,US Dollar,14.630000,US Dollar,Reinvestment,0,NaN,NaN,NaN,NaN,NaN,1.000000,14.630000,1.000000,14.630000
2,2022-09-01,19601,80BFF1CF0,19601,80BFF1CF0,23.639999,US Dollar,23.639999,US Dollar,Reinvestment,0,NaN,NaN,NaN,NaN,NaN,1.000000,23.639999,1.000000,23.639999
3,2022-09-01,319837,80BFF75C0,319837,80BFF75C0,27938.070312,US Dollar,27938.070312,US Dollar,Reinvestment,0,NaN,NaN,NaN,NaN,NaN,1.000000,27938.070312,1.000000,27938.070312
4,2022-09-01,1688,80C007CD0,2843,80073DA70,47.810001,US Dollar,47.810001,US Dollar,Credit Card,0,NaN,NaN,NaN,NaN,NaN,1.000000,47.810001,1.000000,47.810001


In [ ]:
df_augmented.shape

(5078280, 20)

In [ ]:
len(df_augmented[df_augmented['Is Laundering']==1])

5177

Check:

For the LI-Small_Trans.csv data card: 3.565 frauds should be present
For the HI-Small_Trans.csv data card: 5.177 frauds should be present.

# Test for creating the dataframe sender receiver behaviour sender receiver

In [4]:
data = df_augmented

del df_augmented

In [ ]:
# Create a set of sender-receiver pairs
unique_accounts = set(data['Account']).union(set(data['Account.1']))
print(len(unique_accounts))


515070


In [ ]:
# Step 2: Convert to a list and take the first 20
selected_account = list(unique_accounts)[:20]
print(selected_account)

['812FC0560', '8085756B0', '81181AD00', '80EBB8DA0', '806924600', '804C98120', '811919130', '805118B90', '805F024F0', '81091F3C0', '800AA40D0', '80D471DA0', '802CE8720', '802AC7370', '80B91C510', '80019EC50', '80DF96F20', '80237B8C0', '80117D700', '8022E75F0']


The final_df consinder only the 20 accounts in the selected_account list.

In [ ]:
filtered_transactions = data[
    (data['Account'].isin(selected_account)) |
    (data['Account.1'].isin(selected_account))
]

In [ ]:
# This data contains only 20 accounts from selected accounts
filtered_transactions

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,type_fraud,fraud_id,pattern_number,depth_info,depht_type_info,forex_multiple_Amount Paid,Normaized_Amount Paid,forex_multiple_Amount Received,Normaized_Amount Received
16300,2022-09-01 00:01:00,701,800411880,21258,800DB92A0,23590.570312,US Dollar,23590.570312,US Dollar,Credit Card,0,NaN,NaN,NaN,NaN,NaN,1.000000,23590.570312,1.000000,23590.570312
25476,2022-09-01 00:02:00,11813,800CCEF20,14549,801B60340,11611.500000,US Dollar,11611.500000,US Dollar,Cheque,0,NaN,NaN,NaN,NaN,NaN,1.000000,11611.500000,1.000000,11611.500000
25693,2022-09-01 00:02:00,795,800C4AA40,11813,800CCEF20,622.880005,US Dollar,622.880005,US Dollar,Cheque,0,NaN,NaN,NaN,NaN,NaN,1.000000,622.880005,1.000000,622.880005
33343,2022-09-01 00:03:00,117698,80DD47F70,117698,80DD47F70,2344.810059,Euro,2344.810059,Euro,Reinvestment,0,NaN,NaN,NaN,NaN,NaN,1.003905,2353.966797,1.003905,2353.966797
51015,2022-09-01 00:04:00,21258,800DB92A0,21258,800DB92A0,95001.109375,US Dollar,95001.109375,US Dollar,Reinvestment,0,NaN,NaN,NaN,NaN,NaN,1.000000,95001.109375,1.000000,95001.109375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5059342,2022-09-10 21:44:00,23344,802120C90,2467,8104DEB10,1889.510010,Euro,1889.510010,Euro,Credit Card,0,NaN,NaN,NaN,NaN,NaN,1.001202,1891.780273,1.001202,1891.780273
5059918,2022-09-10 21:48:00,23344,802120C90,2467,8104DEB10,257387.687500,Euro,257387.687500,Euro,Wire,0,NaN,NaN,NaN,NaN,NaN,1.001202,257696.937500,1.001202,257696.937500
5060039,2022-09-10 21:49:00,23344,802120C90,2467,8104DEB10,143375.687500,Euro,143375.687500,Euro,Cheque,0,NaN,NaN,NaN,NaN,NaN,1.001202,143547.953125,1.001202,143547.953125
5060997,2022-09-10 21:56:00,11974,8014E3E10,211050,807595470,962.159973,US Dollar,962.159973,US Dollar,Credit Card,0,NaN,NaN,NaN,NaN,NaN,1.000000,962.159973,1.000000,962.159973


# Creation of the functions necessary for feature engeneering

## Behaviour of sender/receiver as single account
The following functions (account_extractor_behaviour, account_extractor_behaviour_2, account_extractor_behviour_currency) will be capable of computing the following features:

1. statistics - avg, std, min, max for sender as sender and receiver as receiver, for the account in general (sender considering sender/receiver txn, receiver considering sender/receiver txn)
2. time from last transactions for an account (as last time that the account sends or last time the account received, or in general last time the account did a transactions as sender/receiver)
3. SELF LOOP - is sender = receiver?
4. IS THE FIRST TRANSACTION? For sender and receiver overall, for sender is the first transaction that sends and for receiver is the first transaction that received.
5. sender ratio - ratio for an account of being a sender over the total appearance
6. is common - count of transaction a sender do to the same receiver (DA CAPIRE)
7. CURRENCY- number of different currency used, same currency as the txn before? for overall behviour and for sender when sends and receiver when receivs. Softmax of the distribution (?).


Statistics: Receiver + Sender + Both
Time from last
First Transaction: Receiver + Sender + Both

In [ ]:
file_path = r"G:\My Drive\HI_parsed_ibm_aml.pkl.gz"

with gzip.open(file_path, "rb") as f:
    df_augmented = pickle.load(f)

data = df_augmented
del df_augmented

In [5]:
data = data.reset_index()
data.rename(columns={"index": "transaction_id"}, inplace=True)


In [6]:
st = time.time()

receiving_transactions = data[['transaction_id' ,'Account.1', 'Timestamp',
                                                'Normaized_Amount Received']]

receiving_transactions = receiving_transactions.copy()
receiving_transactions.loc[:,'Paid'] = [0] * len(receiving_transactions)
receiving_transactions.columns = ['transaction_id' ,'Account', 'Timestamp',
                                'Amount', 'Paid']


# Shift the 'Amount' column within each group (to exclude the current row)
receiving_transactions['shifted'] = receiving_transactions.groupby('Account')['Amount'].shift(1)

# Compute all expanding statistics in one go using agg
exp_stats = (
    receiving_transactions
    .groupby('Account')['shifted']
    .expanding()
    .agg(['mean', 'std', 'max', 'min'])
    .reset_index(level=0, drop=True)
    .fillna(0)
)



# Assign the computed statistics back to the DataFrame
receiving_transactions['rec_exp_mean'] = exp_stats['mean']
receiving_transactions['rec_exp_std']  = exp_stats['std']
receiving_transactions['rec_exp_max']  = exp_stats['max']
receiving_transactions['rec_exp_min']  = exp_stats['min']

# Optionally, remove the temporary shifted column
receiving_transactions.drop(columns='shifted', inplace=True)




sending_transactions = data[['transaction_id' ,'Account', 'Timestamp',
                                                'Normaized_Amount Paid']]

sending_transactions = sending_transactions.copy()
sending_transactions.loc[:,'Paid'] = [0] * len(sending_transactions)
sending_transactions.columns = ['transaction_id' ,'Account', 'Timestamp',
                                'Amount', 'Paid']


# Shift the 'Amount' column within each group (to exclude the current row)
sending_transactions['shifted'] = sending_transactions.groupby('Account')['Amount'].shift(1)

# Compute all expanding statistics in one go using agg
exp_stats = (
    sending_transactions
    .groupby('Account')['shifted']
    .expanding()
    .agg(['mean', 'std', 'max', 'min'])
    .reset_index(level=0, drop=True)
    .fillna(0)
)


# Assign the computed statistics back to the DataFrame
sending_transactions['sen_exp_mean'] = exp_stats['mean']
sending_transactions['sen_exp_std']  = exp_stats['std']
sending_transactions['sen_exp_max']  = exp_stats['max']
sending_transactions['sen_exp_min']  = exp_stats['min']

# Optionally, remove the temporary shifted column
sending_transactions.drop(columns='shifted', inplace=True)

et = time.time()





The total time taken to compute the statistics is: 106.9743


In [9]:
data.columns

Index(['transaction_id', 'Timestamp', 'From Bank', 'Account', 'To Bank',
       'Account.1', 'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering', 'type_fraud',
       'fraud_id', 'pattern_number', 'depth_info', 'depht_type_info',
       'forex_multiple_Amount Paid', 'Normaized_Amount Paid',
       'forex_multiple_Amount Received', 'Normaized_Amount Received',
       'Pay_m_enc_Saudi Riyal', 'Pay_m_enc_US Dollar', 'Pay_m_enc_Shekel',
       'Pay_m_enc_Brazil Real', 'Pay_m_enc_Bitcoin', 'Pay_m_enc_Swiss Franc',
       'Pay_m_enc_Yen', 'Pay_m_enc_Mexican Peso', 'Pay_m_enc_Euro',
       'Pay_m_enc_Rupee', 'Pay_m_enc_Yuan', 'Pay_m_enc_Ruble',
       'Pay_m_enc_UK Pound', 'Pay_m_enc_Australian Dollar',
       'Pay_m_enc_Canadian Dollar', 'Recv_m_enc_Saudi Riyal',
       'Recv_m_enc_US Dollar', 'Recv_m_enc_Shekel', 'Recv_m_enc_Brazil Real',
       'Recv_m_enc_Bitcoin', 'Recv_m_enc_Swiss Franc', 'Recv_m_enc_Yen',
       'Recv_m_e

In [20]:
overall_transactions = data[['transaction_id' ,'Timestamp',
                               'Account.1','Normaized_Amount Received',
                               'Account','Normaized_Amount Paid']]


overall_transactions = overall_transactions.copy()

In [25]:
overall_transactions_test = pd.concat(
  ( pd.DataFrame({'transaction_id': overall_transactions['transaction_id'],
                  'Amount': overall_transactions['Normaized_Amount Received' ],
                  'Account': overall_transactions['Account'],
                  'Paid': [0] * len(overall_transactions)}),

    pd.DataFrame({'transaction_id': overall_transactions['transaction_id'],
                  'Amount': overall_transactions['Normaized_Amount Paid'],
                  'Account': overall_transactions['Account.1'],
                  'Paid': [1] * len(overall_transactions)}
                 )
    )
)

In [29]:
# Shift the 'Amount' column within each group (to exclude the current row)
overall_transactions['shifted'] = overall_transactions.groupby('Account')['Amount'].shift(1)

# Compute all expanding statistics in one go using agg
exp_stats = (
    overall_transactions
    .groupby('Account')['shifted']
    .expanding()
    .agg(['mean', 'std', 'max', 'min'])
    .reset_index(level=0, drop=True)
    .fillna(0)
)


# Assign the computed statistics back to the DataFrame
overall_transactions['all_exp_mean'] = exp_stats['mean']
overall_transactions['all_exp_std']  = exp_stats['std']
overall_transactions['all_exp_max']  = exp_stats['max']
overall_transactions['all_exp_min']  = exp_stats['min']

# Optionally, remove the temporary shifted column
overall_transactions.drop(columns='shifted', inplace=True)


KeyError: 'Column not found: Amount'

In [ ]:
# Shift the 'Amount' column within each group (to exclude the current row)
receiving_transactions['shifted'] = receiving_transactions.groupby('Account')['Amount'].shift(1)

# Compute all expanding statistics in one go using agg
exp_stats = (
    receiving_transactions
    .groupby('Account')['shifted']
    .expanding()
    .agg(['mean', 'std', 'max', 'min'])
    .reset_index(level=0, drop=True)
    .fillna(0)
)

In [ ]:

# Assign the computed statistics back to the DataFrame
receiving_transactions['exp_mean'] = exp_stats['mean']
receiving_transactions['exp_std']  = exp_stats['std']
receiving_transactions['exp_max']  = exp_stats['max']
receiving_transactions['exp_min']  = exp_stats['min']

# Optionally, remove the temporary shifted column
receiving_transactions.drop(columns='shifted', inplace=True)

In [ ]:
receiving_transactions

,transaction_id,Account,Timestamp,Amount,Paid,exp_mean,exp_std,exp_max,exp_min
0,0,8123FB9B0,2022-09-01 00:00:00,1.269091e+01,0,0.000000,0.0,0.000000,0.000000
1,1,802E97980,2022-09-01 00:00:00,1.463000e+01,0,0.000000,0.0,0.000000,0.000000
2,2,80BFF1CF0,2022-09-01 00:00:00,2.364000e+01,0,0.000000,0.0,0.000000,0.000000
3,3,80BFF75C0,2022-09-01 00:00:00,2.793807e+04,0,0.000000,0.0,0.000000,0.000000
4,4,80073DA70,2022-09-01 00:00:00,4.781000e+01,0,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
99995,99995,8026C91F0,2022-09-01 00:09:00,3.103094e+04,0,20.148376,0.0,20.148376,20.148376
99996,99996,811815F70,2022-09-01 00:09:00,6.011295e-03,0,0.000000,0.0,0.000000,0.000000
99997,99997,8010AA160,2022-09-01 00:09:00,1.284000e+01,0,0.000000,0.0,0.000000,0.000000
99998,99998,81028CC70,2022-09-01 00:09:00,2.132676e+04,0,14817.660156,0.0,14817.660156,14817.660156


In [ ]:
test

,transaction_id,Account,Timestamp,Amount,Paid,exp_mean,exp_std,exp_max,exp_min
9,9,80B931BD0,2022-09-01 00:00:00,11874.660156,0,NaN,NaN,NaN,NaN
36832,36832,80B931BD0,2022-09-01 00:03:00,4129.169922,0,NaN,NaN,NaN,NaN
370281,370285,80B931BD0,2022-09-01 01:47:00,132493.609375,0,NaN,NaN,NaN,NaN
672863,672869,80B931BD0,2022-09-01 10:49:00,233927.843750,0,NaN,NaN,NaN,NaN
676341,676347,80B931BD0,2022-09-01 10:55:00,23439.630859,0,NaN,NaN,NaN,NaN


In [ ]:
receiving_transactions[receiving_transactions['Account'] == '80073DA70']

,transaction_id,Account,Timestamp,Amount,Paid,exp_mean,exp_std,exp_max,exp_min
4,4,80073DA70,2022-09-01,47.810001,0,0.0,0.0,0.0,0.0


In [ ]:
def account_extractor_behaviour(df, selected_account=None):
  """
    Compute sender and receiver statistics (average, std, max, min) for each transaction at a certain time t in the dataframe, considering all past transactions up to time t-1.
    Also computes the time since the last transaction in minutes for each account.
    Those features are computed for both sender and receiver considering the cases when sender(receiver) sends(receives) and overall transactions an account does (as sender or as receivers)

    Parameters:
        df (pd.DataFrame): The dataframe containing transaction data.
        selected_account (list, optional): List of selected accounts. If provided, the dataframe is filtered
                                           to include only transactions involving these accounts.

    Returns:
        pd.DataFrame: The dataframe with additional statistical columns.

 """

    # If selected_account is provided, filter the dataframe to include only transactions involving those accounts.


  if selected_account:

      df = df[df['Account'].isin(selected_account) | df['Account.1'].isin(selected_account)]

  df.sort_values(by='Timestamp')

  # STEP 1: Creation of lists and dictionaries to store results

  # Initialize lists to store computed STATISTICS
  sender_avg_sent, sender_std_sent, sender_max_sent, sender_min_sent = [], [], [], [] # consider past transactions only where sender sends (is sender)
  receiver_avg_received, receiver_std_received, receiver_max_received, receiver_min_received = [], [], [], [] # consider past transactions only where receiver receives (is receiver)
  s_avg_txn, s_std_txn, s_max_txn, s_min_txn = [], [], [], [] # consider all sender past transactions
  r_avg_txn, r_std_txn, r_max_txn, r_min_txn = [], [], [], [] # consider all receiver past transactions

  # Initialize list to store TIME SINCE LAST TRANSACTION (tsl) for each account
  time_since_last_sender_txn = [] # time since last transaction for sender when sends
  time_since_last_receiver_txn = [] # time since last transaction for receiver when receives
  tsl_txn_s, tsl_txn_r = [], [] # time since last transaction for sender and receiver considering all past transactions

  # Initialize list for FIRST TRANSACTIONS
  first_transaction_sender = [] # first overall transaction sender
  first_transaction_receiver = [] # first overall transaction receiver
  first_transactions_s_sent = [] # first transaction that sender sends
  first_transactions_r_received = [] # first transaction that receiver receives

  # Dictionaries to store historical transaction data
  sender_sent_history = {}  # Stores Normaized_Amount Paid (value) for each sender (key), store sender when sent history
  receiver_received_history = {}  # Stores Normaized_Amount Received (value) for each receiver (key), store receiver when received history
  account_txn_history = {} # Stores Normaized_Amount Paid and Received

  # Dictionaries to store the last transaction time for each account and sender-receiver pair
  last_sender_txn_time = {}  # Last transaction time (value) for each sender (key) when sent
  last_receiver_txn_time = {}  # Last transaction time (value) for each receiver (key) when received
  last_transactions_time = {} # Last transaction time (value) for each account (key)


  # Iterate through each transaction in chronological order
  for idx, row in df.iterrows():

      # STEP 2: Get info about the current transaction
      timestamp = row['Timestamp']
      sender = row['Account']
      receiver = row['Account.1']
      sent_amount = row['Normaized_Amount Paid']
      received_amount = row['Normaized_Amount Received']

      # STEP 3: Feature Engineering - TIMELASPSE since last transaction in MINUTES

      # step 3.1 - considering sender only when sends and receiver only when receives
      # Calculate the time since last transaction for sender when sents
      time_since_last_sender = (timestamp - last_sender_txn_time.get(sender, timestamp)).total_seconds() / 60
      time_since_last_sender_txn.append(time_since_last_sender)
      #print(time_since_last_sender_txn, last_sender_txn_time.get(sender, timestamp))

      # Calculate the time since last transaction for receiver when receives
      time_since_last_receiver = (timestamp - last_receiver_txn_time.get(receiver, timestamp)).total_seconds() / 60
      time_since_last_receiver_txn.append(time_since_last_receiver)
      #print(time_since_last_receiver_txn, last_receiver_txn_time.get(receiver, timestamp))

      # Update last transaction times
      last_sender_txn_time[sender] = timestamp
      last_receiver_txn_time[receiver] = timestamp
      #print(last_sender_txn_time)

      # step 3.2 - considering all past transactions of an account
      # Calculate the time since last transaction for sender
      time_since_last_sender = (timestamp - last_transactions_time.get(sender, timestamp)).total_seconds() / 60
      tsl_txn_s.append(time_since_last_sender)
      #print(tsl_txn_s, last_transactions_time.get(sender, timestamp))
      #print(last_transactions_time)

      # Calculate the time since last transaction for receiver
      time_since_last_receiver = (timestamp - last_transactions_time.get(receiver, timestamp)).total_seconds() / 60
      tsl_txn_r.append(time_since_last_receiver)

      # Update last transaction times
      last_transactions_time[sender] = timestamp
      last_transactions_time[receiver] = timestamp


      # STEP 4: Feature Engineering: STATISTICS since last transaction (avg, std, max, min)

      # step 4.1 - consdiering only sender transactions when sent and receiver transactions when receives
      # Compute sender account when sent statistics
      sender_history = sender_sent_history.get(sender, [])
      s_avg_sent = np.mean(sender_history) if sender_history else 0 # avg
      s_std_sent = np.std(sender_history) if sender_history else 0  # std
      s_max_sent = max(sender_history) if sender_history else 0   # max
      s_min_sent = (min(sender_history) if sender_history else 0)    # min
      sender_avg_sent.append(s_avg_sent)
      sender_std_sent.append(s_std_sent)
      sender_max_sent.append(s_max_sent)
      sender_min_sent.append(s_min_sent)
      sender_sent_history.setdefault(sender, []).append(sent_amount) # Update transaction history records
      #print(sender_history)

      # Compute receiver account when receives statistics
      receiver_history = receiver_received_history.get(receiver, [])
      r_avg_received = np.mean(receiver_history) if receiver_history else 0 # avg
      r_std_received = np.std(receiver_history) if receiver_history else 0 # std
      r_max_received = max(receiver_history) if receiver_history else 0  # max
      r_min_received = min(receiver_history) if receiver_history else 0  # min
      receiver_avg_received.append(r_avg_received)
      receiver_std_received.append(r_std_received)
      receiver_max_received.append(r_max_received)
      receiver_min_received.append(r_min_received)
      receiver_received_history.setdefault(receiver, []).append(received_amount) # Update transaction history records
      #print(receiver_received_history)

      # step 4.2 - considering all past transactions of an account
      # Calculate sender statistics
      s_avg = np.mean(account_txn_history.get(sender, [])) if sender in account_txn_history else 0
      s_std = np.std(account_txn_history.get(sender, [])) if sender in account_txn_history else 0
      s_max = max(account_txn_history.get(sender, [])) if sender in account_txn_history else 0
      s_min = min(account_txn_history.get(sender, [])) if sender in account_txn_history else 0

      # Append to lists
      s_avg_txn.append(s_avg)
      s_std_txn.append(s_std)
      s_max_txn.append(s_max)
      s_min_txn.append(s_min)

      # Compute receiver statistics
      r_avg = np.mean(account_txn_history.get(receiver, [])) if receiver in account_txn_history else 0
      r_std = np.std(account_txn_history.get(receiver, [])) if receiver in account_txn_history else 0
      r_max = max(account_txn_history.get(receiver, [])) if receiver in account_txn_history else 0
      r_min = min(account_txn_history.get(receiver, [])) if receiver in account_txn_history else 0

      # Append to lists
      r_avg_txn.append(r_avg)
      r_std_txn.append(r_std)
      r_max_txn.append(r_max)
      r_min_txn.append(r_min)

      # Update transaction history - handle self-transactions
      if sender == receiver:
            # For self-transactions, only add once
            if sender not in account_txn_history:
                account_txn_history[sender] = [sent_amount]
            elif account_txn_history[sender][-1] != sent_amount:  # Optional: avoid duplicates
                account_txn_history[sender].append(sent_amount)
      else:
            # Normal case - different sender and receiver
            account_txn_history.setdefault(sender, []).append(sent_amount)
            account_txn_history.setdefault(receiver, []).append(received_amount)


    # STEP 5: Feature engeneering - IS THE FIRST TRANSACTION?
    # Check if this is the first OVERALL transaction for sender or receiver
      # Check if sender has no prior transactions (all stats = 0)
      sum_s_stats = s_avg + s_std + s_max + s_min
      first_transaction_sender.append(1 if sum_s_stats == 0 else 0)
      # Check if receiver ha no prior transactions (all stats = 0)
      sum_r_stats = r_avg + r_std + r_max + r_min
      first_transaction_receiver.append(1 if sum_r_stats == 0 else 0)

    # Check if this is the first transactions
    # Check if is the first sender transaction that sends (all stats = 0)
      sum_s_stats_sent = s_avg_sent + s_std_sent + s_max_sent + s_min_sent
      first_transactions_s_sent.append(1 if sum_s_stats_sent == 0 else 0)
      # Check if is the first receiver transaction that receives (all stats = 0)
      sum_r_stats_received = r_avg_received + r_std_received + r_max_received + r_min_received
      first_transactions_r_received.append(1 if sum_r_stats_received == 0 else 0)

# STEP 6: Add calculated features as columns to the dataframe
  # Time since last transactions feature
  df['tsl_s_txn_sent'] = time_since_last_sender_txn # time since last transaction sender when sent
  df['tsl_r_txn_received'] = time_since_last_receiver_txn # time since last transaction receiver when receives
  df['tsl_s_txn'] = tsl_txn_s # time since last transaction sender
  df['tsl_r_txn'] = tsl_txn_r # time since last transaction receiver

  # Statistics feature
  # Statistics for sender when sent
  df['s_avg_sent'] = sender_avg_sent
  df['s_std_sent'] = sender_std_sent
  df['s_max_sent'] = sender_max_sent
  df['s_min_sent'] = sender_min_sent

  # Statistics for sender all past transactions
  df['s_avg_txn'] = s_avg_txn
  df['s_std_txn'] = s_std_txn
  df['s_max_txn'] = s_max_txn
  df['s_min_txn'] = s_min_txn

  # Statistics for receiver when receives
  df['r_avg_received'] = receiver_avg_received
  df['r_std_received'] = receiver_std_received
  df['r_max_received'] = receiver_max_received
  df['r_min_received'] = receiver_min_received

  # Statistics for receiver all past transactions
  df['r_avg_txn'] = r_avg_txn
  df['r_std_txn'] = r_std_txn
  df['r_max_txn'] = r_max_txn
  df['r_min_txn'] = r_min_txn

  # First transaction coloumns
  df['first_overall_s_txn'] = first_transaction_sender
  df['first_overall_r_txn'] = first_transaction_receiver
  df['first_s_txn_sent'] = first_transactions_s_sent
  df['first_r_txn_received'] = first_transactions_r_received

  return df
  """
    [['Timestamp', 'Account', 'Account.1', 'Normaized_Amount Paid',
             'tsl_s_txn_sent', 'tsl_r_txn_received',
             'tsl_s_txn', 'tsl_r_txn',
             's_avg_sent','s_std_sent', 's_max_sent', 's_min_sent',
             's_avg_txn', 's_std_txn', 's_max_txn', 's_min_txn',
             'r_avg_received', 'r_std_received', 'r_max_received', 'r_min_received',
             'r_avg_txn', 'r_std_txn', 'r_max_txn', 'r_min_txn',
             'first_overall_s_txn', 'first_overall_r_txn', 'first_s_txn_sent', 'first_r_txn_received'
             ]]
  """

In [ ]:
account_extractor_behaviour(data,selected_account=selected_account) # selected_account are 20 unique selected account

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,...,r_max_received,r_min_received,r_avg_txn,r_std_txn,r_max_txn,r_min_txn,first_overall_s_txn,first_overall_r_txn,first_s_txn_sent,first_r_txn_received
16300,2022-09-01 00:01:00,701,800411880,21258,800DB92A0,23590.570312,US Dollar,23590.570312,US Dollar,Credit Card,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,1,1,1
25476,2022-09-01 00:02:00,11813,800CCEF20,14549,801B60340,11611.500000,US Dollar,11611.500000,US Dollar,Cheque,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,1,1,1
25693,2022-09-01 00:02:00,795,800C4AA40,11813,800CCEF20,622.880005,US Dollar,622.880005,US Dollar,Cheque,...,0.000000,0.000000,11611.500000,0.000000,11611.500000,11611.500000,1,0,1,1
33343,2022-09-01 00:03:00,117698,80DD47F70,117698,80DD47F70,2344.810059,Euro,2344.810059,Euro,Reinvestment,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,1,1,1
51015,2022-09-01 00:04:00,21258,800DB92A0,21258,800DB92A0,95001.109375,US Dollar,95001.109375,US Dollar,Reinvestment,...,23590.570312,23590.570312,23590.570312,0.000000,23590.570312,23590.570312,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5059342,2022-09-10 21:44:00,23344,802120C90,2467,8104DEB10,1889.510010,Euro,1889.510010,Euro,Credit Card,...,258392.828125,217.428528,113853.759969,92887.397691,258392.828125,217.428528,0,0,0,0
5059918,2022-09-10 21:48:00,23344,802120C90,2467,8104DEB10,257387.687500,Euro,257387.687500,Euro,Wire,...,258392.828125,217.428528,111122.979976,93359.061294,258392.828125,217.428528,0,0,0,0
5060039,2022-09-10 21:49:00,23344,802120C90,2467,8104DEB10,143375.687500,Euro,143375.687500,Euro,Cheque,...,258392.828125,217.428528,114612.836108,94909.090632,258392.828125,217.428528,0,0,0,0
5060997,2022-09-10 21:56:00,11974,8014E3E10,211050,807595470,962.159973,US Dollar,962.159973,US Dollar,Credit Card,...,83699.171875,9.430000,3655.254910,12783.700954,83699.171875,9.430000,0,0,0,0


In [ ]:
st = time.time()
account_behaviour = account_extractor_behaviour(data)
et = time.time()
print(f"Time taken: {et - st} seconds")


KeyboardInterrupt: 

In [ ]:
def account_extractor_behaviour_2(df, selected_account=None):
    """
    This second function for account behavior consider if sender = receiver (self loop) creating a new column self_loop assigning 1 if there is a self loop and 0 if not.
    Also compute the sender_ratio_sender and sender_ratio_receiver that are the ratio of an account sender/receiver of being a sender in past transactions (up to time t-1).
    Parameters:
        df (pd.DataFrame): The dataframe containing transaction data.
        selected_account (list, optional): List of selected accounts. If provided, the dataframe is filtered
                                           to include only transactions involving these accounts.

      Returns:
          pd.DataFrame: The dataframe with additional statistical columns.
   """
    if selected_account:
        df = df[df['Account'].isin(selected_account) | df['Account.1'].isin(selected_account)]
    df.sort_values(by='Timestamp')

    # STEP 1: Creation of lists and dictionaries to store results
    # Initilize list for SELF-LOOP (is sender = receiver?) and SENDER RATIO
    self_loop = [] # list of 0,1
    sender_ratio_sender= [] # list sender ratio for sender account (sender_count/total appearance)
    sender_ratio_receiver = [] # list sender ratio for receiver account (sender_count/total appearance)

    # Dictionaries to store historical transaction data
    sender_count = {} # dictionary that has as key the account and as value the count of the account being a sender
    total_appearances = {} #  dictionary that has as key the account and as value the count of the account appearances

    # Iterate through each transaction in chronological order
    for idx, row in df.iterrows():

        # STEP 2: Get information of the current transaction
        sender = row['Account']
        receiver = row['Account.1']
        is_self_loop = (sender == receiver)

        current_sender_count = sender_count.get(sender, 0)
        current_sender_total = total_appearances.get(sender, 0)

        # STEP 3: FEATURE ENGENEERING computing SENDER RATIO for both sender and receiver
        # When there is a self loop (sender = receiver), the account appearances is counted as 1 and counted as 1 the fact that the account send
        if current_sender_total > 0:
            sender_ratio_sender.append(current_sender_count / current_sender_total)
        else:
            sender_ratio_sender.append(0.0)

        if not is_self_loop:
            current_receiver_total = total_appearances.get(receiver, 0)
            current_receiver_count = sender_count.get(receiver, 0)
            if current_receiver_total > 0:
                sender_ratio_receiver.append(current_receiver_count / current_receiver_total)
            else:
                sender_ratio_receiver.append(0.0)
        else:
            sender_ratio_receiver.append(0.0)

        sender_count[sender] = current_sender_count + 1
        if is_self_loop:
            total_appearances[sender] = current_sender_total + 1
        else:
            total_appearances[sender] = current_sender_total + 1
            total_appearances[receiver] = total_appearances.get(receiver, 0) + 1


      # STEP 4: FEATURE ENGENEERING - is a self loop?
      # Check if this is a self-loop transaction (1 = YES, 0 = NO)
        is_a_self_loop = 1 if sender == receiver else 0
        self_loop.append(is_a_self_loop)


    # STEP 5: Add calculated features as columns to the dataframe
    df['sender_ratio_s'] = sender_ratio_sender # sender ratio of the sender
    df['sender_ratio_r'] = sender_ratio_receiver # sender ratio of the receiver
    df['self_loop'] = self_loop

    return df
    """
     [['Timestamp','Account','Account.1',
               'sender_ratio_s', 'sender_ratio_r',
               'self_loop'
               ]]
    """

In [ ]:
account_extractor_behaviour_2(data,selected_account=selected_account)

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,...,pattern_number,depth_info,depht_type_info,forex_multiple_Amount Paid,Normaized_Amount Paid,forex_multiple_Amount Received,Normaized_Amount Received,sender_ratio_s,sender_ratio_r,self_loop
9017,2022-09-01 00:00:00,27637,805118B90,27637,805118B90,8.120000,US Dollar,8.120000,US Dollar,Reinvestment,...,NaN,NaN,NaN,1.000000,8.120000,1.000000,8.120000,0.000000,0.000000,1
19289,2022-09-01 00:01:00,143598,81091F3C0,143598,81091F3C0,70.699997,Shekel,70.699997,Shekel,Reinvestment,...,NaN,NaN,NaN,0.300565,21.249929,0.300565,21.249929,0.000000,0.000000,1
39654,2022-09-01 00:03:00,2843,80117D700,2843,80117D700,20.070000,US Dollar,20.070000,US Dollar,Reinvestment,...,NaN,NaN,NaN,1.000000,20.070000,1.000000,20.070000,0.000000,0.000000,1
40965,2022-09-01 00:03:00,2843,80117D700,12446,8077DEB60,14844.990234,US Dollar,14844.990234,US Dollar,Credit Card,...,NaN,NaN,NaN,1.000000,14844.990234,1.000000,14844.990234,1.000000,0.000000,0
41529,2022-09-01 00:03:00,220271,80DF96F20,220271,80DF96F20,6193.759766,US Dollar,6193.759766,US Dollar,Reinvestment,...,NaN,NaN,NaN,1.000000,6193.759766,1.000000,6193.759766,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995359,2022-09-10 13:45:00,701,8005E0F80,21940,802CE8720,1672.400024,US Dollar,1672.400024,US Dollar,Credit Card,...,NaN,NaN,NaN,1.000000,1672.400024,1.000000,1672.400024,1.000000,0.615385,0
5043055,2022-09-10 19:44:00,21940,802CE8720,16224,80404C2C0,3178.760010,US Dollar,3178.760010,US Dollar,Cheque,...,NaN,NaN,NaN,1.000000,3178.760010,1.000000,3178.760010,0.606061,0.000000,0
5043735,2022-09-10 19:49:00,21940,802CE8720,16224,80404C2C0,1551.369995,US Dollar,1551.369995,US Dollar,Cash,...,NaN,NaN,NaN,1.000000,1551.369995,1.000000,1551.369995,0.611940,0.000000,0
5058556,2022-09-10 21:37:00,3514,8085756B0,11,8148378D0,128.229996,Euro,128.229996,Euro,ACH,...,NaN,NaN,NaN,1.001202,128.384064,1.001202,128.384064,0.689655,0.000000,0


In [ ]:
# 6 minutes
st = time.time()
account_behaviour_2 = account_extractor_behaviour_2(data)
et = time.time()
print(f"Time taken: {et - st} seconds")

Time taken: 365.41678524017334 seconds


In [ ]:
def account_extractor_behaviour_currency(df, selected_account=None):
    """
      This function takes in consideration Payment Currency and Receiving Currency. It creates new features that consider if an account sender/receiver chenges is currency w.r.t. previous transactions
      (0=no, 1=yes and still 0 if it's the first transactions -> THIS IS SOMETHING TO DISCUSS)
      Also the function give new funters that rapresent the count of differet currency a sender/receiver had in past transactions, both in the case consider sender(receiver) sending (receiving) and
      overall transactions they made.

      Parameters:
          df (pd.DataFrame): The dataframe containing transaction data.
          selected_account (list, optional): List of selected accounts. If provided, the dataframe is filtered
                                             to include only transactions involving these accounts.
    """
    if selected_account:
        df = df[df['Account'].isin(selected_account) | df['Account.1'].isin(selected_account)]
    df.sort_values(by='Timestamp')


    # STEP 1: Creation of lists and dictionaries to store results
    # Creation of lists
    s_currency_count_sent, r_currency_count_received = [], [] # currency count for sender(receiver) when sending(receiving)
    s_currency_count_overall, r_currency_count_overall = [], [] # currency count for sender(receiver) considering overall past transactions
    s_currency_change_sent, r_currency_change_received = [], [] # currency change for sender(receiver) when sending(receiving)
    s_currency_change_overall, r_currency_change_overall = [], [] # currency change for sender(receiver) considering overall past transactions

    # Creation of dictionaries
    sender_currencies, receiver_currencies, currencies = {}, {}, {} # sender(receiver) currencies and currencies of both sender and receiver where the currency is the value and the account is the key
    last_sender_currency, last_receiver_currency, last_currency_overall = {}, {}, {} # last currency of sender(receiver) when sending (receiving) and last currency overall

    # Iterate through each transaction in chronological order
    for idx, row in df.iterrows():
        sender = row['Account']
        receiver = row['Account.1']
        sent_currency = row['Payment Currency']
        received_currency = row['Receiving Currency']
        is_self_loop = (sender == receiver)


        # STEP 2: FEATURE ENGENEERING getting CURRENCY COUNT (default to empty set if missing)
        s_currency_count_sent.append(len(sender_currencies.get(sender, set())))
        r_currency_count_received.append(len(receiver_currencies.get(receiver, set())))
        s_currency_count_overall.append(len(currencies.get(sender, set())))
        r_currency_count_overall.append(len(currencies.get(receiver, set())))

        # Update currency sets
        # Only sender when sending
        if sender not in sender_currencies:
            sender_currencies[sender] = set()
        sender_currencies[sender].add(sent_currency)
        # Only receiver when receiving
        if receiver not in receiver_currencies:
            receiver_currencies[receiver] = set()
        if not is_self_loop or sent_currency != received_currency:
            receiver_currencies[receiver].add(received_currency)
        #print(sender_currencies)
        # Sender in overall past transactions
        if sender not in currencies:
            currencies[sender] = set()
        if not is_self_loop or sent_currency != received_currency:
            currencies[sender].add(sent_currency)
        # Receiver in overall past transactions
        if receiver not in currencies:
            currencies[receiver] = set()
        if not is_self_loop or sent_currency != received_currency:
            currencies[receiver].add(received_currency)


        # STEP 3: FEATURE ENGENEERING - IS CURRENCY CHANGED? YES=1, NO=0, FIRST TRANSACTION (no past info)=0 (NEED TO BE MANAGED)
         # Check for sender currency change

        last_currency_s_sent = last_sender_currency.get(sender)
        last_currency_r_received = last_receiver_currency.get(receiver)
        last_currency_overall_s = last_currency_overall.get(sender)
        last_currency_overall_r = last_currency_overall.get(receiver)

        c_changed_sent = 1 if (last_currency_s_sent is not None and last_currency_s_sent != sent_currency) else 0
        c_changed_received = 1 if (last_currency_r_received is not None and last_currency_r_received != received_currency) else 0
        c_changed_overall_s = 1 if (last_currency_overall_s is not None and last_currency_overall_s != sent_currency) else 0
        c_changed_overall_r = 1 if (last_currency_overall_r is not None and last_currency_overall_r != received_currency) else 0

        s_currency_change_sent.append(c_changed_sent)
        r_currency_change_received.append(c_changed_received)
        s_currency_change_overall.append(c_changed_overall_s)
        r_currency_change_overall.append(c_changed_overall_r)

        last_sender_currency[sender] = sent_currency
        last_receiver_currency[receiver] = received_currency
        last_currency_overall[sender] = sent_currency
        last_currency_overall[receiver] = received_currency


    # STEP 4: Creating new columns
    df['s_currency_count_sent'] = s_currency_count_sent
    df['r_currency_count_received'] = r_currency_count_received
    df['s_currency_count_overall'] = s_currency_count_overall
    df['r_currency_count_overall'] = r_currency_count_overall

    df['s_currency_change_sent'] = s_currency_change_sent
    df['r_currency_change_received'] = r_currency_change_received
    df['s_currency_change_overall'] = s_currency_change_overall
    df['r_currency_change_overall'] = r_currency_change_overall

    return df
    """
      [['Timestamp', 'Account', 'Account.1', 'Normaized_Amount Paid',
               'Payment Currency', 'Receiving Currency',
               's_currency_count_sent', 'r_currency_count_received',
               's_currency_count_overall', 'r_currency_count_overall',
               's_currency_change_sent', 'r_currency_change_received',
               's_currency_change_overall', 'r_currency_change_overall'
               ]]
     """

In [ ]:
account_extractor_behaviour_currency(data,selected_account=selected_account)

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,...,sender_ratio_r,self_loop,s_currency_count_sent,r_currency_count_received,s_currency_count_overall,r_currency_count_overall,s_currency_change_sent,r_currency_change_received,s_currency_change_overall,r_currency_change_overall
9017,2022-09-01 00:00:00,27637,805118B90,27637,805118B90,8.120000,US Dollar,8.120000,US Dollar,Reinvestment,...,0.000000,1,0,0,0,0,0,0,0,0
19289,2022-09-01 00:01:00,143598,81091F3C0,143598,81091F3C0,70.699997,Shekel,70.699997,Shekel,Reinvestment,...,0.000000,1,0,0,0,0,0,0,0,0
39654,2022-09-01 00:03:00,2843,80117D700,2843,80117D700,20.070000,US Dollar,20.070000,US Dollar,Reinvestment,...,0.000000,1,0,0,0,0,0,0,0,0
40965,2022-09-01 00:03:00,2843,80117D700,12446,8077DEB60,14844.990234,US Dollar,14844.990234,US Dollar,Credit Card,...,0.000000,0,1,0,0,0,0,0,0,0
41529,2022-09-01 00:03:00,220271,80DF96F20,220271,80DF96F20,6193.759766,US Dollar,6193.759766,US Dollar,Reinvestment,...,0.000000,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995359,2022-09-10 13:45:00,701,8005E0F80,21940,802CE8720,1672.400024,US Dollar,1672.400024,US Dollar,Credit Card,...,0.615385,0,1,1,1,1,0,0,0,0
5043055,2022-09-10 19:44:00,21940,802CE8720,16224,80404C2C0,3178.760010,US Dollar,3178.760010,US Dollar,Cheque,...,0.350000,0,1,1,1,1,0,0,0,0
5043735,2022-09-10 19:49:00,21940,802CE8720,16224,80404C2C0,1551.369995,US Dollar,1551.369995,US Dollar,Cash,...,0.341463,0,1,1,1,1,0,0,0,0
5058556,2022-09-10 21:37:00,3514,8085756B0,11,8148378D0,128.229996,Euro,128.229996,Euro,ACH,...,0.048780,0,1,1,1,1,0,0,0,0


In [ ]:
# 7 minutes
st = time.time()
account_behaviour_currency = account_extractor_behaviour_currency(data)
et = time.time()
print(f"Time taken: {et - st} seconds")

Time taken: 448.4173791408539 seconds


In [ ]:
account_behaviour_currency

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,...,sender_ratio_r,self_loop,s_currency_count_sent,r_currency_count_received,s_currency_count_overall,r_currency_count_overall,s_currency_change_sent,r_currency_change_received,s_currency_change_overall,r_currency_change_overall
0,2022-09-01 00:00:00,121,8123FB9B0,121,8123FB9B0,47.639999,Saudi Riyal,47.639999,Saudi Riyal,Reinvestment,...,0.000000,1,0,0,0,0,0,0,0,0
1,2022-09-01 00:00:00,2454,802E97980,2454,802E97980,14.630000,US Dollar,14.630000,US Dollar,Reinvestment,...,0.000000,1,0,0,0,0,0,0,0,0
2,2022-09-01 00:00:00,19601,80BFF1CF0,19601,80BFF1CF0,23.639999,US Dollar,23.639999,US Dollar,Reinvestment,...,0.000000,1,0,0,0,0,0,0,0,0
3,2022-09-01 00:00:00,319837,80BFF75C0,319837,80BFF75C0,27938.070312,US Dollar,27938.070312,US Dollar,Reinvestment,...,0.000000,1,0,0,0,0,0,0,0,0
4,2022-09-01 00:00:00,1688,80C007CD0,2843,80073DA70,47.810001,US Dollar,47.810001,US Dollar,Credit Card,...,0.000000,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5078340,2022-09-18 10:02:00,9371,8043A0FB0,16163,802F78670,3749.139893,US Dollar,3749.139893,US Dollar,ACH,...,0.793103,0,4,2,4,2,0,0,0,0
5078341,2022-09-18 11:18:00,9371,8043A0FB0,9371,8043A0FB0,1785.270020,Euro,2091.949951,US Dollar,ACH,...,0.000000,1,4,4,4,4,0,0,0,1
5078342,2022-09-18 11:18:00,9371,8043A0FB0,13858,8095526B0,1785.270020,Euro,1785.270020,Euro,ACH,...,1.000000,0,4,0,4,0,1,0,0,0
5078343,2022-09-18 12:58:00,9371,8043A0FB0,1124,8026687E0,2154.540039,US Dollar,2154.540039,US Dollar,ACH,...,0.736842,0,4,1,4,1,1,0,1,0


In [ ]:
# Tis pickle file contain the result of account_extractor_behviour_2 and accoun_extractor_currency
with gzip.open('/content/drive/My Drive/account_behaviour_currency.pkl.gz', 'wb') as f:
    pickle.dump(account_behaviour_currency, f)

# Behaviour of the pair sender-receiver
The following function (pair_extractor_behaviour) will be able to compute the following features for each pair of account sender/receiver:

1. Statistics (avg, std, max, min)
2. Time since last transactions
3. Is the first transactions?
4. Payment format the pair used: count, different from last transactions, softmax
5. Are the two account belong to the same Bank?


N.B. We are considering as pair A-B and B-A at time t and what is the history of the pair till time t-1.

In [ ]:
def pair_extractor_behaviour(df, selected_account=None):
  """
    The function create feature taking into consideration the PAIR of account and their transaction history (till time t-1). Therefore, it computes statistics
    (avg, std, min, max), time since last transaction, is the first transactions, are the two accounts from the same bank.
    Also consider some features linked with Payment Format such as count of different payment format for each pair, is the payment format change w.r.t. previous transaction.

    Parameters:
          df (pd.DataFrame): The dataframe containing transaction data.
          selected_account (list, optional): List of selected accounts. If provided, the dataframe is filtered
                                             to include only transactions involving these accounts.
  """
  if selected_account:
        df = df[df['Account'].isin(selected_account) | df['Account.1'].isin(selected_account)]
  df.sort_values(by='Timestamp')

  # STEP 1: Creation of lists and dictionaries to store results
    # Creation of lists
  pair_avg_amount, pair_std_amount, pair_max_amount, pair_min_amount = [], [], [], [] # statistics for sender-receiver as a pair
  tsl_pair_txn = [] # time since last transactions for the pair
  first_pair_txn = [] # is the first transaction

  pair_payment_format_counts = [] # list of counts of different payment format
  current_payment_format = [] # list with the current payment format
  payment_format_changed = [] # list of 0,1 if the payment format is change or not

  same_bank_status = [] # list of 0,1 if the bank is the same for the two account ot not

    # Creation of dictionaries
  sender_receiver_history = {} # key the pair and value the amount over time
  last_txn_time = {} # key the pair and value time
  pair_payment_formats = {} # key the pair and value the payment format of that pair
  last_payment_format = {} # key the pair and value the last payment format in time

  # Iterate through each transaction in chronological order
  for idx, row in df.iterrows():

      # STEP 2: Get info about the current transaction
      timestamp = row['Timestamp']
      sender = row['Account']
      receiver = row['Account.1']
      from_bank = row['From Bank']
      to_bank = row['To Bank']
      sent_amount = row['Normaized_Amount Paid']  # Consider only sent amount for sender behavior
      received_amount = row['Normaized_Amount Received']  # Consider only received amount for receiver behavior
      payment_format = row['Payment Format']

      pair = tuple(sorted([sender, receiver]))  # Ensure (A, B) and (B, A) are treated the same

      # STEP 3: TIME SINCE LAST TRANSACTIONS for the pair
      time_since_last_sender_receiver = (timestamp - last_txn_time.get(pair, timestamp)).total_seconds() / 60
      tsl_pair_txn.append(time_since_last_sender_receiver)

      last_txn_time[pair] = timestamp

      # STEP 4: FEATURE ENGENEERING - creating the STATISTICS for the pair
      pair_history = sender_receiver_history.get(pair, [])
      pair_avg = np.mean(pair_history) if pair_history else 0
      pair_std = np.std(pair_history) if pair_history else 0
      pair_max = max(pair_history) if pair_history else 0
      pair_min = min(pair_history) if pair_history else 0

      pair_avg_amount.append(pair_avg)
      pair_std_amount.append(pair_std)
      pair_max_amount.append(pair_max)
      pair_min_amount.append(pair_min)

      sender_receiver_history.setdefault(pair, []).append(sent_amount)

      # step 4.1 - IS THE FIRST PAIR TRANSACTIONS? 0 = NO, 1 = YES
      sum_stats = pair_avg + pair_std + pair_max + pair_min
      first_pair_txn.append(1 if sum_stats == 0 else 0)


      # STEP 5: FEATURE ENGENEERING - looking at PAYMENT FORMAT distribution
      # Count different payment format for each pair
      current_payment_format.append(payment_format)
      format_history = pair_payment_formats.get(pair, set())
      pair_payment_format_counts.append(len(format_history))
      pair_payment_formats.setdefault(pair, set()).add(payment_format)
      #print(pair_payment_formats)

      # Is the payment format changed since last transaction of the pair? 0 = NO, 1 = YES and 0 is also first transaction (no info before)
      last_format = last_payment_format.get(pair)
      format_changed = 1 if (last_format is not None and last_format != payment_format) else 0
      payment_format_changed.append(format_changed)

      last_payment_format[pair] = payment_format
      #print(last_payment_format)


      # STEP 6: FEATURE ENGEENERING - are the two account in the pair in the same bank?
      if from_bank == to_bank:
            same_bank_status.append(1)  # Same bank Yes = 1
      else:
            same_bank_status.append(0)  # Different banks No = 0


      #print(last_payment_format)

  # STEP 7: Creating the new columns
  df['pair_payment_format_count'] = pair_payment_format_counts
  df['payment_format_changed'] = payment_format_changed
  df['pair_avg_amount'] = pair_avg_amount
  df['pair_std_amount'] = pair_std_amount
  df['pair_max_amount'] = pair_max_amount
  df['pair_min_amount'] = pair_min_amount
  df['tsl_pair_txn'] = tsl_pair_txn
  df['first_pair_txn'] = first_pair_txn
  df['pair_same_bank_status'] = same_bank_status

  return df
  """
    [['Timestamp','Account','Account.1','Normaized_Amount Paid',
               'pair_avg_amount', 'pair_std_amount', 'pair_max_amount', 'pair_min_amount',
               'tsl_pair_txn', 'first_pair_txn',
                'Payment Format','pair_payment_format_count', 'payment_format_changed',
                'pair_same_bank_status', 'From Bank','To Bank'
                ]]
   """




In [ ]:
pair_extractor_behaviour(data, selected_account=selected_account)

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,...,Normaized_Amount Received,pair_payment_format_count,payment_format_changed,pair_avg_amount,pair_std_amount,pair_max_amount,pair_min_amount,tsl_pair_txn,first_pair_txn,pair_same_bank_status
16300,2022-09-01 00:01:00,701,800411880,21258,800DB92A0,23590.570312,US Dollar,23590.570312,US Dollar,Credit Card,...,23590.570312,0,0,0.000000,0.000000,0.000000,0.000000,0.0,1,0
25476,2022-09-01 00:02:00,11813,800CCEF20,14549,801B60340,11611.500000,US Dollar,11611.500000,US Dollar,Cheque,...,11611.500000,0,0,0.000000,0.000000,0.000000,0.000000,0.0,1,0
25693,2022-09-01 00:02:00,795,800C4AA40,11813,800CCEF20,622.880005,US Dollar,622.880005,US Dollar,Cheque,...,622.880005,0,0,0.000000,0.000000,0.000000,0.000000,0.0,1,0
33343,2022-09-01 00:03:00,117698,80DD47F70,117698,80DD47F70,2344.810059,Euro,2344.810059,Euro,Reinvestment,...,2353.966797,0,0,0.000000,0.000000,0.000000,0.000000,0.0,1,1
51015,2022-09-01 00:04:00,21258,800DB92A0,21258,800DB92A0,95001.109375,US Dollar,95001.109375,US Dollar,Reinvestment,...,95001.109375,0,0,0.000000,0.000000,0.000000,0.000000,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5059342,2022-09-10 21:44:00,23344,802120C90,2467,8104DEB10,1889.510010,Euro,1889.510010,Euro,Credit Card,...,1891.780273,4,1,122951.734949,90684.793936,258392.828125,1870.227783,12.0,0,0
5059918,2022-09-10 21:48:00,23344,802120C90,2467,8104DEB10,257387.687500,Euro,257387.687500,Euro,Wire,...,257696.937500,4,1,119765.946668,91557.850527,258392.828125,1870.227783,4.0,0,0
5060039,2022-09-10 21:49:00,23344,802120C90,2467,8104DEB10,143375.687500,Euro,143375.687500,Euro,Cheque,...,143547.953125,4,1,123302.638741,92968.847626,258392.828125,1870.227783,1.0,0,0
5060997,2022-09-10 21:56:00,11974,8014E3E10,211050,807595470,962.159973,US Dollar,962.159973,US Dollar,Credit Card,...,962.159973,2,0,846.104980,116.054993,962.159973,730.049988,1768.0,0,0


QUESTION
pair_payment_format_counts put 0 if there is no information of previous transactions, is it right?
The point is to mantain the temporality order otherwise those features does not make sense.

In [ ]:
def softmax(x):
    """Compute softmax values for a vector."""
    exp_x = np.exp(x - np.max(x))  # Subtract max for numerical stability
    return exp_x / exp_x.sum()


def payment_softmax_pair(df, selected_account=None):


  if selected_account:
        df = df[df['Account'].isin(selected_account) | df['Account.1'].isin(selected_account)]
  df.sort_values(by='Timestamp')


  # Get unique payment formats
  unique_formats = sorted(df['Payment Format'].unique())  # Keep consistent order
  format_to_index = {fmt: i for i, fmt in enumerate(unique_formats)}

    # Initialize softmax matrix
  payment_format_softmax = np.zeros((len(df), len(unique_formats)))


  payment_format_softmax = []

  for idx, row in df.iterrows():
    timestamp = row['Timestamp']
    sender = row['Account']
    receiver = row['Account.1']
    from_bank = row['From Bank']
    to_bank = row['To Bank']
    sent_amount = row['Normaized_Amount Paid']  # Consider only sent amount for sender behavior
    received_amount = row['Normaized_Amount Received']  # Consider only received amount for receiver behavior
    payment_format = row['Payment Format']
    pair = tuple(sorted([sender, receiver]))  # Ensure (A, B) and (B, A) are treated the same

    one_hot_vector = np.zeros(len(unique_formats))
    one_hot_vector[format_to_index[payment_format]] = 1

        # Apply softmax
    payment_format_softmax.append(softmax(one_hot_vector))

    # Convert softmax values into separate columns without changing the index
  softmax_df = pd.DataFrame(payment_format_softmax,
                              columns=[f'softmax_{fmt}' for fmt in unique_formats],
                              index=df.index)  # Keep original index

    # Merge softmax values into df while preserving its index
  df = pd.concat([df, softmax_df], axis=1)

  return df[['Timestamp','Account','Account.1','Normaized_Amount Paid',
               'Payment Format'] +
               [f'softmax_{fmt}' for fmt in unique_formats]]


In [ ]:
data[(data['Account']=='8001DA790') | (data['Account.1']=='8001DA790')].sort_values(by='Timestamp')

Considering all accounts and transactions.

In [ ]:
st = time.time()
all = account_extractor_behaviour(all_final_df)
et = time.time()
print(f"Time taken: {et - st} seconds")

Time taken: 1530.3846085071564 seconds


23 minuti e 55 secondi

In [ ]:
all

,Timestamp,Account,Account.1,Normaized_Amount Paid,Normaized_Amount Received,Total_Amount,sender_avg_amount,receiver_avg_amount,sender_receiver_avg_amount
0,2022-09-01 00:00:00,8123FB9B0,8123FB9B0,12.690906,12.690906,25.381811,0.000000,0.000000,0.000000
7313,2022-09-01 00:00:00,800D4E490,800D4E750,13993.484375,13993.484375,27986.968750,0.000000,0.000000,0.000000
7314,2022-09-01 00:00:00,800C8D9D0,800C8D9D0,10.410497,10.410497,20.820993,0.000000,0.000000,0.000000
7315,2022-09-01 00:00:00,800C71170,800C71170,6693.578125,6693.578125,13387.156250,0.000000,0.000000,0.000000
7316,2022-09-01 00:00:00,8140A8A71,8140A8A71,21.294516,21.294516,42.589031,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
5078340,2022-09-18 10:02:00,8043A0FB0,802F78670,3749.139893,3749.139893,7498.279785,9331.811908,17449.463887,0.000000
5078341,2022-09-18 11:18:00,8043A0FB0,8043A0FB0,2091.949951,1783.183594,3875.133545,9201.982326,964518.256119,13485.725065
5078342,2022-09-18 11:18:00,8043A0FB0,8095526B0,1783.183594,1783.183594,3566.367188,9040.390682,101829.031250,0.000000
5078343,2022-09-18 12:58:00,8043A0FB0,8026687E0,2154.540039,2154.540039,4309.080078,8879.119413,2946.453967,0.000000


In [ ]:
# prompt: create pickle file

with gzip.open('/content/drive/My Drive/all_transactions_statistics_2.pkl.gz', 'wb') as f:
    pickle.dump(all, f)
